# Introduction

Sudoku is a logic-based puzzle that originated in Japan.  It is appealing to people because the level of difficulty can be chosen by the user.  The levels of difficulty range from super easy to difficult.  Acceptance grew rapidly once it was introduced outside of Japan, partially because upon a quick look at a Sudoku puzzle, one can easily confuse it with a crossword puzzle (see figure 1).	

A typical Sudoku puzzle consists of a 9x9 grid that features the numbers 1-9.  As shown in figure 1, there is a main box around the 9x9 grid, but there are mini boxes of size 3x3, as seen in the grid.	

In order to win, a user must follow specific rules.  The main rules are:
* The user can only use the numbers 1 to 9.
* Every mini box (grid size 3x3) can only contain the numbers 1 to 9.
* Every mini box cannot have duplicate numbers.	
* Every vertical column can only contain the numbers 1 to 9.
* Every vertical column cannot have duplicate numbers.	
* Every horizontal row can only contain the numbers 1 to 9.
* Every horizontal row cannot have duplicate numbers.

Failure to follow the rules will result in duplicate numbers in either a mini box, a vertical column, or a horizontal row.  If the rules are followed, then the Sudoku puzzle is solved.	


# Proposed Approach	

My proposed approach is to use the Sudoku rules mentioned in the introduction.  Looking at all the rules at the same time can be stressing, but if I decompose the problems into smaller ones, then the problem is approachable as follows:	

Let $\textbf{ψ}$ represent all the columns, such that $\textbf{ψ_i}$ represents the $\textbf{i^{th}}$ column	
Let $\textbf{ϕ}$ represent all the rows, such that $\textbf{ϕ_j}$ represents the $\textbf{j^{th}}$ row	
Let $\textbf{φ}$ represent all the mini boxes, such that $\textbf{φ_k}$ represents the $\textbf{k^{th}}$ mini box	

Having assigned the columns, rows, and mini boxes to symbols, I can now enforce the rules presented in the introduction.  To verify, for example, that $\textbf{ψ_1}$ holds, I would need to verify that for every number in $\textbf{ψ_1}$ that there are no duplicates. I would then need to perform the same test on $\textbf{ψ_2…ψ_9}$.  After performing that test, $\textbf{ψ}$ will hold if and only if  $\textbf{ψ_1∧ψ_2∧ψ_3∧ψ_4∧ψ_5∧ψ_6∧ψ_7∧ψ_8∧ψ_9}$ holds.  The same will hold true for $\textbf{ϕ}$ (all the rows) and for $\textbf{φ}$ (all the mini boxes).  

Following this logic, if $\textbf{ψ∧ϕ∧φ}$ holds, then all the rules have been followed and the user has solve the Sudoku puzzle.

While I have previously completed a Sudoku solver using dynamic programing, this time I plan on using a satisfiability module theory (SMT) solver, specifically the Z3 SMT solver.  This will allow me to apply the knowledge learned from class and to learn to program in the Z3 environment.	


$\Phi$ $\Psi$  
$$\phi_1$$

### Required Imports

In [1]:
from z3 import *
from datetime import datetime
import numpy as np
import pickle

### Configuration

In [41]:
size = 25

# Restraints (from introduction)

As a recap, here are the restraints that my Sudoku Z3 solver requires

* <span style="color:green">Every mini box (grid size 3x3) can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 1</span>)
* Every mini box cannot have duplicate numbers.  
* <span style="color:green">Every vertical column can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 2</span>)
* Every vertical column cannot have duplicate numbers.  
* <span style="color:green">Every horizontal row can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 3</span>)
* Every horizontal row cannot have duplicate numbers.  
* <span style="color:green">Every mini box can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 4</span>)  
* Every box cannot have duplicate numbers.  

### Solution

We first set the solution to restrict every element (shown in <span style="color:green">green above</span>).

We'll first create integer references for all locations, $L$, in the form $L_{ij}$ where $i$ refers to the rows and $j$ refers to the columns.

In [42]:
L = []
for row in range(size):
    L.append( [Int(f'L_{row}{col}') for col in range(size)] )

Next, we create the first restraint: a conjunction for every location ($L_{ij}$) where $1 \leq L_{ij} \leq 9$, then we check the constraints, such that  
`constraint_1` = 
$$\begin{align}
&L_{00} \wedge L_{01} \wedge L_{02} \wedge L_{03} \wedge L_{04} \wedge L_{05} \wedge L_{06} \wedge L_{07} \wedge L_{08} \wedge \\
&L_{10} \wedge L_{11} \wedge L_{12} \wedge L_{13} \wedge L_{14} \wedge L_{15} \wedge L_{16} \wedge L_{17} \wedge L_{18} \wedge \\
&L_{20} \wedge L_{21} \wedge L_{22} \wedge L_{23} \wedge L_{24} \wedge L_{25} \wedge L_{26} \wedge L_{27} \wedge L_{28} \wedge \\
&L_{30} \wedge L_{31} \wedge L_{32} \wedge L_{33} \wedge L_{34} \wedge L_{35} \wedge L_{36} \wedge L_{37} \wedge L_{38} \wedge \\
&L_{40} \wedge L_{41} \wedge L_{42} \wedge L_{43} \wedge L_{44} \wedge L_{45} \wedge L_{46} \wedge L_{47} \wedge L_{48} \wedge \\
&L_{50} \wedge L_{51} \wedge L_{52} \wedge L_{53} \wedge L_{54} \wedge L_{55} \wedge L_{56} \wedge L_{57} \wedge L_{58} \wedge \\
&L_{60} \wedge L_{61} \wedge L_{62} \wedge L_{63} \wedge L_{64} \wedge L_{65} \wedge L_{66} \wedge L_{67} \wedge L_{68} \wedge \\
&L_{70} \wedge L_{71} \wedge L_{72} \wedge L_{73} \wedge L_{74} \wedge L_{75} \wedge L_{76} \wedge L_{77} \wedge L_{78} \wedge \\
&L_{80} \wedge L_{81} \wedge L_{82} \wedge L_{83} \wedge L_{84} \wedge L_{85} \wedge L_{86} \wedge L_{87} \wedge L_{88}
\end{align}$$

In [43]:
constraint_1 = [ And( L[row][col] >= 1, L[row][col] <= size)  for col in range(size) for row in range(size) ]

We now create the second restraint: every vertical column cannot have duplicate numbers, meaning that each vertical column can only contain distinct values. To accomplish this, we create an array of arrays, where each inner array represents a conjunction of each column, such as  
`constraint_2` =  
$$
\text{Column 1}: L_{00} \wedge L_{10} \wedge \cdots \wedge L_{70} \wedge L_{80} \\
\text{Column 2}: L_{01} \wedge L_{11} \wedge \cdots \wedge L_{71} \wedge L_{81} \\
\vdots \hspace{3.5cm} \vdots \hspace{1.5cm} \\
\text{Column 8}: L_{07} \wedge L_{17} \wedge \cdots \wedge L_{77} \wedge L_{87} \\
\text{Column 9}: L_{08} \wedge L_{18} \wedge \cdots \wedge L_{78} \wedge L_{88} \\
$$

In [44]:
constraint_2 = []
for col in range(size):
    constraint_2.append( Distinct( [L[row][col] for row in range(size)] ) )

We now create the third restraint: every horizontal row cannot have duplicate numbers, meaning that each horizontal row can only contain distinct values. To accomplish this, we create an array of arrays, where each inner array represents a conjunction of each row, such as  
`constraint_3` =  
$$
\text{Row 1}: L_{00} \wedge L_{01} \wedge \cdots \wedge L_{07} \wedge L_{08} \\
\text{Row 2}: L_{10} \wedge L_{11} \wedge \cdots \wedge L_{17} \wedge L_{18} \\
\vdots \hspace{3.25cm} \vdots \hspace{1.90cm} \\
\text{Row 8}: L_{70} \wedge L_{71} \wedge \cdots \wedge L_{77} \wedge L_{78} \\
\text{Row 9}: L_{88} \wedge L_{88} \wedge \cdots \wedge L_{87} \wedge L_{88} \\
$$

In [45]:
constraint_3 = []
for row in range(size):
    constraint_3.append( Distinct( [L[row][col] for col in range(size)] ) )

We now create the fourth restraint: every mini box cannot have duplicate numbers, meaning that every mini box can only contain distinct values.  To accomplish this, we create helper a helper function, that given an $ij$ location for a mini box, will return a distinct mini box, such as
`constraint_4` =  

<img src="picture.jpg" alt="Paris" width="500px" class="center">

In [46]:
def get_mini_grid(array, i, j, size):
    return [ L[i * size + y][j * size + x] for y in range(size) for x in range(size) ]

mini_size = size // int(size**0.5)
print(f'{mini_size=}')

constraint_4 = [ Distinct( get_mini_grid(L, y, x, mini_size) )  for y in range(mini_size) for x in range(mini_size) ]  

mini_size=5


### Create The Solver

Now we create the solver and add the restraints

In [47]:
s = Solver()
s.add(constraint_1)
s.add(constraint_2)
s.add(constraint_3)
s.add(constraint_4)

In [48]:
puzzle = (
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,3,0,0,0,13,0,0,0,4),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,8),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,7,0,0,0,0,0,0,0,0,12,0,0,0),
)

Copy the Sudoku puzzle corresponding to every location in $L_{ij}$ and add them as constraint.  If a number is added, then it's a literal, otherwise it's left open for the Z3 solver to solve.

In [49]:
# puzzle = (
#     (0,7,2,0,0,0,1,0,0),
#     (9,0,4,0,0,0,0,0,6),
#     (0,0,0,0,7,5,0,0,2),
#     (0,3,6,0,0,4,0,0,0),
#     (4,0,0,0,9,0,0,0,3),
#     (0,0,0,3,0,0,6,1,0),
#     (3,0,0,9,8,0,0,0,0),
#     (7,0,0,0,0,0,3,0,9),
#     (0,0,8,0,0,0,2,4,0)
# )

# puzzle = (
#     (0,0,0,0,0,2,8,0,3),
#     (0,0,0,0,0,0,0,1,0),
#     (5,0,9,3,1,0,0,2,4),
#     (7,0,0,0,5,3,0,0,0),
#     (0,0,0,7,0,9,0,0,0),
#     (0,0,0,4,2,0,0,0,1),
#     (9,1,0,0,3,7,5,0,8),
#     (0,4,0,0,0,0,0,0,0),
#     (8,0,3,9,0,0,0,0,0)
# )

# puzzle = (
#     (0,0,0,0,0,0,0,0,0),
#     (0,0,0,0,0,0,0,0,0),
#     (0,0,0,0,0,0,0,1,0),
#     (0,0,0,0,0,0,0,0,0),
#     (0,0,0,0,0,0,0,0,0),
#     (0,0,0,0,0,0,0,0,0),
#     (0,0,0,0,0,0,0,0,0),
#     (0,0,0,9,0,0,0,0,0),
#     (0,0,0,0,0,0,0,0,0),
# )

file2 = open(r'pickle.dump', 'rb')
puzzle = pickle.load(file2)
file2.close()

puzzle

# constraint_5 = [ If(puzzle[i][j] == 0, True, L[i][j] == puzzle[i][j]) for i in range(9) for j in range(9) ]
constraint_5 = [L[i][j] == puzzle[i][j] for i in range(size) for j in range(size) if puzzle[i][j]]
s.add(constraint_5)

In [50]:
[' '.join([f'{z:>2}' for z in map(str,_)]) for _ in puzzle]

[' 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0 24  0  0  0  0  0',
 ' 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  7  0  0  0  0  0  0  0',
 ' 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  0  0 16  0  0  0 22  0  0  0  0  0 18  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  0  0  0  0  0  0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  0  0  0  0 24  0  0  0  0  0  0  0  0  0  0  0  0 21  1  0  0  0  0',
 ' 0  0  0  0  0  0  0  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 11  0  0  0',
 ' 0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0',
 ' 0  0  0  0  0  0  0  0  0  0 23  0  0  0  0  0  0

Finally we check if the model is SAT or UNSAT.  If the model is UNSAT, then we mention it; otherwise, we display the solution.

In [51]:
def get_grid(puzzler):
    size = len(puzzler)
    m_size = int(size**(1/2))
    char_len = len(str(size))
    horizontal = '+' + ''.join([f'{"-"*(m_size * (char_len + 1) + 1)}+' for _ in range(m_size)])
    output = ''
    for r in range(size):
        if r % m_size == 0:
            output = output + f'{horizontal}\n'
        line = [' '.join([f'{_:>{char_len}}' for _ in puzzler[r][(c * m_size):(c * m_size+ m_size)]]) 
                for c in range(0, m_size)]
        line_with_divider = ' | '.join([_ for _ in line])
        output = output + f'| {line_with_divider} |\n'
    return f'{output}{horizontal}'.replace(f'{0:>{char_len}}',f'{"∙":>{char_len}}')

def side_by_side(grid1,grid2, spaces=8):
    gs1, gs2 = get_grid(grid1), get_grid(grid2)
    s = [f'{s1}{" "*8}{s2}' for (s1, s2) in zip(gs1.split('\n'), gs2.split('\n'))]
    return '\n'.join(s)

def exec_time(start, end):
    return str(end - start)

def get_solution(model, L):
    #return [ [ model.eval(L[i][j]) for j in range(size) ] for i in range(size) ]
    return [ [ model.eval(L[i][j]).as_long() for j in range(size) ] for i in range(size) ]

In [52]:
start = datetime.now()
status = str(s.check())
end = datetime.now()

print('Execution time:', exec_time(start, end), '\n')

solution = 'Model is UNSAT' if status == 'unset' else get_solution(s.model(), L)

if status == 'unsat':
    print(solution)
else:
    print(get_grid(solution))
    #print(side_by_side(puzzle, solution).replace(' 0', ' '))

Execution time: 0:01:29.206645 

+----------------+----------------+----------------+----------------+----------------+
|  7 21 18  2  3 |  1 12 24  9  6 | 11 16 13 15 14 | 22 17 19  8  5 | 20 25  4 10 23 |
| 22 25 14  9 17 | 10  7 15 16  2 |  1  8 12 18  4 | 21 20  6 23 24 | 19  3  5 13 11 |
| 10 13 15 12  5 |  3 11  4 19 23 | 25  6 21 20 24 |  9  1  7  2 14 | 18 17 22 16  8 |
| 16 19 24  8  6 | 22 18 20 13 21 | 17  3  5 10 23 | 12 11  4 15 25 |  7 14  9  2  1 |
| 23  1  4 11 20 | 17  8 25 14  5 | 19  7  9 22  2 | 10  3 18 16 13 | 24 21 15 12  6 |
+----------------+----------------+----------------+----------------+----------------+
| 24 10  8  6 16 |  7 21 11 22  3 |  9 12 20  1 18 | 15  4 17 14 19 |  5 13 23 25  2 |
|  2 22 11 15  7 | 19  4  1 18  9 |  5 21 25  8 17 | 23 13  3 10  6 | 14 20 12 24 16 |
| 20  9  1 25 21 |  8  6 14  5 17 |  3  2 23 16 13 | 18 22 24 11 12 | 10 15  7  4 19 |
|  5 18  3 19 23 | 16 24 13 10 12 | 15  4 11 14 22 |  7  2 20 25 21 |  1  8 17  6  9 |
|  4 12 13

In [15]:
status

'unsat'

In [ ]:
puzzlez = (
    (0,0,0,0,0,2,8,0,3),
    (0,0,0,0,0,0,0,1,0),
    (5,0,9,3,1,0,0,2,4),
    (7,0,0,0,5,3,0,0,0),
    (0,0,0,7,0,9,0,0,0),
    (0,0,0,4,2,0,0,0,1),
    (9,1,0,0,3,7,5,0,8),
    (0,4,0,0,0,0,0,0,0),
    (8,0,3,9,0,0,0,0,0)
)

p = []
for line in puzzlez:
    length, cl = len(line), list(line)
    left, index = 25 - length, length
    while left > 0:
        cl.append(line[index % length])
        index = index + 1
        left = left - 1
    p.append(cl)

index = 0
left = len(p[0]) - len(p)
length = len(p[0])
while left > 0:
    p.append(p[index % length])
    left = left - 1
    index = index + 1
print('\n'.join([str(_) for _ in p]))

In [ ]:
print(get_grid(p))

In [ ]:
puzzlez = (
    (0,0,0,0,0,2,8,0,3,),
    (0,0,0,0,0,0,0,1,0),
    (5,0,9,3,1,0,0,2,4),
    (7,0,0,0,5,3,0,0,0),
    (0,0,0,7,0,9,0,0,0),
    (0,0,0,4,2,0,0,0,1),
    (9,1,0,0,3,7,5,0,8),
    (0,4,0,0,0,0,0,0,0),
    (8,0,3,9,0,0,0,0,0)
)
print(get_grid(puzzlez))

In [ ]:
sol = np.array([ [solution[i][j].as_long() for j in range(size)] for i in range(size)])
sol

In [ ]:
np.sum(sol, axis=0)

In [ ]:
sol.sum(axis=1)

In [ ]:
sol.sum(axis=1) == [sum(range(1,size+1))] * size

In [ ]:
g = [[1,2,3,4],
     [1,2,3,4],
     [1,2,3,4],
     [1,2,3,4]]

t = [1,2,3,4]

gp = np.array(g)

In [ ]:
gp

In [ ]:
gp == t

In [ ]:
np.all(gp==t, axis=1)

In [ ]:
np.unique(gp, axis=1)

In [ ]:
gp

In [ ]:
set(gp[0])

In [ ]:
len(set(gp[:,2]))

In [ ]:
gp.T

In [ ]:
[len(set(_)) for i in size for _ in (gp[:,i] if axis else gp[i])]

In [ ]:
axis=1
z = [len(set(_)) for _ in (gp if axis == 1 else gp.T)]
z

In [ ]:
siz = 4
z2 = [siz]*siz
np.all(z == z2)

In [ ]:
def distinct(grid, axis=1):
    '''
    Verifies that either each of the columns or the rows has unique values. The check is based on the axis: axis=1, the default, checks the distinctness of the rows, whereas axis=0 checks the distinctness of the columns.
    '''
    if (axis > 2 or axis < 0):
        raise Exception(f'Invalid value for axis: {axis=}')
    if axis == 2:
        grid_len = len(grid)
        mini_box = int(math.sqrt(grid_len))
        grid_prime = [grid[(i*mini_box):(i*mini_box + mini_box), (j*mini_box):(j*mini_box + mini_box)].flatten() 
                         for i in range(mini_box) for j in range(mini_box)]
        lengths = [len(set(_)) for _ in grid_prime]
        grid_len = len(grid)
        grid_len = [grid_len] * grid_len
        return np.all(lengths == grid_len)
    
    lengths = [len(set(_)) for _ in (grid if axis == 1 else grid.T)]
    grid_len = len(grid)
    grid_len = [grid_len] * grid_len
    return np.all(lengths == grid_len)

In [ ]:
distinct(gp, axis=3)

In [ ]:
?Exception

In [ ]:
math.sqrt(16)

In [ ]:
f = [[1,2,3],[4,5,6],[7,8,9]]

In [ ]:
j = np.array(f).flatten()
k = len(set(j))
l = len(j)
k == l, k, l

In [ ]:
size = 16
mini_box = 4
solution

In [ ]:
grid = np.array(solution)
grid

In [ ]:
i = 3
j = 3
grid[(i*mini_box):(i*mini_box + mini_box), (j*mini_box):(j*mini_box + mini_box)]

In [ ]:
[grid[(i*mini_box):(i*mini_box + mini_box), (j*mini_box):(j*mini_box + mini_box)].flatten() 
     for i in range(mini_box) for j in range(mini_box)]

In [ ]:
sol = np.array(solution)

In [ ]:
distinct(sol, axis=0)

In [ ]:
distinct(sol, axis=1)

In [ ]:
distinct(sol, axis=2)

In [ ]:
puzzle

In [ ]:
size = 16
m_size = int(size**(1/2))
size, m_size

In [ ]:
top = puzzle[0][:size]
puzzler = [_[:size] for _ in puzzle[:size]] 
puzzler

In [ ]:
 ' | '.join([' '.join(map(str,top[(j*m_size):(j*m_size+m_size)])) for j in range(0, m_size)])

In [ ]:
'+' + ''.join([f'{"-"*(m_size * 2 + 1)}+' for _ in range(m_size)])

In [ ]:
def g_grid(puzzler):
    size = len(puzzler)
    m_size = int(size**(1/2))
    char_len = len(str(size))
    horizontal = '+' + ''.join([f'{"-"*(m_size * (char_len + 1) + 1)}+' for _ in range(m_size)])
    output = ''
    for i in range(size):
        if i % m_size == 0:
            output = output + f'{horizontal}\n'
        line = [' '.join([f'{_:>{char_len}}' for _ in puzzler[i][(j*m_size):(j*m_size+m_size)]]) for j in range(0, m_size)]
        line_with_divider = ' | '.join([_ for _ in line])
        output = output + f'| {line_with_divider} |\n'
    return f'{output}{horizontal}'.replace('0','∙')

In [ ]:
print(g_grid(puzzler))

In [ ]:
f"| {' | '.join([' '.join(map(str,puzzler[i][(j*m_size):(j*m_size+m_size)])) for j in range(0, m_size)])} |\n"

In [ ]:
# output + [' '.join([f'{_:>2}' for _ in puzzler[i][(j*m_size):(j*m_size+m_size)]]) for j in range(0, m_size)] + "\n"

In [ ]:
length

In [ ]:
len(puzzler)